In [3]:
import requests
import selenium
import pandas as pd
import schedule
import threading
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.expected_conditions import presenceOfElementLocated
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException      
import time
import re
import bs4

In [114]:
# shopee_password =  'Qw3rty123'
# shopee_account =  'testpurposes63@gmail.com'
# shopee_username = 'testpurpose154'

BASE_URL = 'https://www.tokopedia.com/'
KEYWORD = 'sony'
PAGE=1
BASE_SEARCH_URL_PARAMETER = f'{BASE_URL}search?st=&q={KEYWORD}'
PROXY_SERVER = '34.82.224.175:33333'
# 15.204.161.192:18080
# 34.82.224.175:33333
# 154.58.202.47:1337
# 34.142.51.21:443
# 64.225.4.17:10005
print(BASE_SEARCH_URL_PARAMETER)

https://www.tokopedia.com/search?st=&q=sony


In [7]:
def print_message(text, color, bold=False):
    color_code = ''
    if color == 'danger':
        color_code = "\033[91m"
    elif color == 'success':
        color_code = "\033[92m" 
    elif color == 'info':
        color_code = "\033[94m"
    elif color == 'warning':
        color_code = "\033[93m"
    
    if bold == True:
        return print(f"{color_code}\033[1m{text}\033[0m")
    else:
        return print(f"{color_code}{text}\033[0m")

In [294]:
## H E L P E R

def scrollFromToptoBottom(dvr, boundary_component, byId=False, no_scroll_top=False):
    scrolled = False
    timeout_scroll = 30
    scroll_count = 0
    is_boundary_component = False
    
    print_message(f'scrolling . . . {boundary_component}', 'info')
#     and
    while not scrolled and scroll_count < timeout_scroll and not is_boundary_component: 
        current_scroll_position = dvr.execute_script("return window.pageYOffset;")
        document_height = dvr.execute_script("return document.documentElement.scrollHeight;")
        print('current_scroll_position', current_scroll_position)
        print('document_height', document_height)
        try:
#             wait = WebDriverWait(dvr, 10)
#             wait.until(EC.presence_of_element_located((By.XPATH, f"//div[@class='{boundary_component}']")))
            boundary_element = dvr.find_element(by=By.CLASS_NAME, value=f'{boundary_component}') if not byId else dvr.find_element(by=By.ID, value=f'{boundary_component}')  
            print('boundary_component', boundary_element)
            if boundary_element:
                scrolled = True
                is_boundary_component = True
        except NoSuchElementException:
            scrolled = False
            is_boundary_component = False
            scroll_height = 500
            dvr.execute_script(f"window.scrollBy(0, {scroll_height});")
            scroll_count += 1
            print('scroll_count', scroll_count)
            time.sleep(5)
        
    if is_boundary_component or scroll_count == timeout_scroll and not no_scrolled_top:
        dvr.execute_script("window.scrollTo(0, 0);")
        return [True]
    if is_boundary_component or scroll_count == timeout_scroll and no_scrolled_top:
        return [True]
    else:
        return [False]
    
def findActiveButtons(array, attr):
    for item in array:
        print('itemmmmm', item)
        if item is not None:
            print('get_attribute', item.get_attribute(f'{attr}'))
            if attr and item.get_attribute(attr) == 'true':
                return item.text
        return None
    
def getReviewDetail(dvr):
    print('MANTAIN REVIEW')
#     wait = WebDriverWait(dvr, 120)

#     element = wait.until(EC.presence_of_element_located((By.XPATH, "//div[@class='css-jxzr5i']")))
#     element1 = wait.until(EC.presence_of_element_located((By.XPATH, "//p[@class='css-e84n4s-unf-heading e1qvo2ff8']")))
#     wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'css-72zbc4')))
#     print('element', element.is_displayed())
#     print('element1', element1.is_displayed())
    time.sleep(60)
    detail_content = dvr.page_source
    detail_soup = BeautifulSoup(detail_content, 'html.parser')

    total_reviews_element = detail_soup.find('p', class_='css-e84n4s-unf-heading e1qvo2ff8')
    current_rows_of_reviewers = detail_soup.find_all('article', class_='css-72zbc4')
    print('current_rows_of_reviewers LENGTH', len(current_rows_of_reviewers))
    print('current_rows_x', current_rows_of_reviewers)
    if len(current_rows_of_reviewers) > 0:
        total_review_string = total_reviews_element.text
        print('total_reviews_element', string)
        total_review = total_review_string.split()[3] if total_review_string else None
        print('TOTAL_REVIEW', total_review)
        if total_review is not None:
            FIND_REVIEW_PAGE_CONTROLLER = detail_soup.find('div', class_='css-1xqkwi8')
            print('FIND_REVIEW_PAGE_CONTROLLER', FIND_REVIEW_PAGE_CONTROLLER)
            navigation_controller = ''
            if FIND_REVIEW_PAGE_CONTROLLER:
                navigation_controller = detail_soup.find('nav', class_='css-txlndr-unf-pagination')
            else:
                navigation_controller = None
            print('navigation_controller', navigation_controller)

            find_total_button_controller = navigation_controller.find_all('button', class_='css-bugrro-unf-pagination-item') if navigation_controller is not None else None
            find_total_button_controller_element = dvr.find_elements(By.XPATH, '//button[@class="css-bugrro-unf-pagination-item"]') if navigation_controller is not None else None
            print('find_total_button_controller_find_total_button_controller', find_total_button_controller[0] if navigation_controller is not None else 'kosong')
            print('FIND_TOTAL_BUTTON_PAGE', find_total_button_controller[-1].text if find_total_button_controller is not None else '')      
            review_count = 0
            review_list = []
            current_button = None
            if find_total_button_controller is not None and find_total_button_controller_element is not None:
                current_button =  findActiveButtons(find_total_button_controller_element, 'data-active')
            else:
                current_button = None
                
            print('current_button', current_button)
            while review_count < int(total_review):
                for review in current_rows_of_reviewers:
                    print('reviewwww', review)
                    customer_name = review.find('span', class_='name').text
                    print('customer_name', customer_name)
                    customer_review_container = review.find('p', class_='css-ed1s1j-unf-heading e1qvo2ff8')
                    print('customer_review_container', customer_review_container)
                    customer_review = customer_review_container.find('span').text
                    print('customer_review', customer_review)

                    customer_review_item = {
                        'customer_name': customer_name,
                        'customer_review': customer_review
                    }
                    review_list.append(customer_review_item)
                    print('review_count', review_count)
                    review_count += 1
            print_message(f'REVIEW - LIST {review_list}', 'info', True)
    

def getSellerDetail(dvr):
    wait = WebDriverWait(dvr, 10)
    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'css-1wbjdax')))
    detail_content = dvr.page_source
    detail_soup = BeautifulSoup(detail_content, 'html.parser')

    seller_container = detail_soup.find('div', class_='css-1wbjdax')
    seller_name = detail_soup.find('div', class_='css-k008qs')
    seller_image = detail_soup.find('img', class_='css-ebxddb')
    detail_soup = BeautifulSoup(detail_content, 'html.parser')
    seller_name = detail_soup.find('h2', class_='css-1wdzqxj-unf-heading e1qvo2ff2').text 
    seller_performance = detail_soup.find_all('div', class_='css-1h5fp8g')
    seller_responsiveness = seller_performance[1].find('span').text
    seller_rate = seller_performance[0].find('span').text
    seller_location_element = detail_soup.find('h2', class_='css-1pd07ge-unf-heading e1qvo2ff2')
    seller_location = seller_location_element.find('b')
#     seller_category_image = detail_soup.find('img', class_='css-ebxddb')
#     print('seller_category_image', seller_category_image)
    seller_category = dvr.find_element(by=By.CLASS_NAME, value='css-ebxddb').get_attribute('alt')
#     seller_category = seller_category_image.get_attribute('alt') if seller_category_image is not None else ''

    seler_section = {
        'seller_name' : seller_name,
        'seller_responsiveness': seller_responsiveness,
        'seller_rate': seller_rate,
        'seller_category': seller_category,
        'seller_location': seller_location
    }
    print_message(f'SELLER - SECTION {seler_section}', 'info', True )

def getProductDetail(dvr):
    detail_content = dvr.page_source
    detail_soup = BeautifulSoup(detail_content, 'html.parser')
    time.sleep(10)
#     print('detail_soup', detail_soup)
    #   PRODUCT - SECTION
    product_name = detail_soup.find('h1', class_='css-1os9jjn').text
    product_category = detail_soup.find_all('li', class_='css-d5bnys')
    product_category_index = product_category[1].text
#     print('product_category', product_category_index)
    product_price_container = detail_soup.find('div', class_='css-chstwd')
    product_original_price = product_price_container.find('div', class_='original-price').text if(product_price_container.find('div', class_='original-price')) else ''
    product_price = product_price_container.find('div', class_='price').text
    scroll_height = 500
    dvr.execute_script(f"window.scrollBy(0, {scroll_height});")
    time.sleep(10)
    #   SELLER - SECTION
    getSellerDetail(dvr)
    scroll_height2 = 1000
    dvr.execute_script(f"window.scrollBy(0, {scroll_height2});")
    getReviewDetail(dvr)
#    

#     print('PRODUCT - SECTION', [product_name, product_category_index, product_original_price, product_price])
#     print_message(f'seler_section {seler_section}', 'info', True)
#     print('seller_category', seller_category.get_text())
# #     CUSTOMER - SECTION

   
    
    

In [220]:
MAX_RETRY = 10
debug_detail_soup = ''
def scrapeTokopediaData():
    print_message(f'SCRAPE FUNCTION RUNNING ON PROXY {PROXY_SERVER} . . .', 'success', bold=True)
    chrome_options = webdriver.ChromeOptions()
    services = Service('chromedriver.exe')
    chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36')
    chrome_options.add_argument('--proxy-server={}'. format(PROXY_SERVER))
    print_message(f'ARGUMENT_ADDED', 'info')
    time.sleep(10)
    driver = webdriver.Chrome(service=services, options=chrome_options)
    print_message(f'DRIVER_SETUP', 'info')
    requestURL = driver.get(BASE_SEARCH_URL_PARAMETER)
    time.sleep(30)
    print_message(f'REQUESTING_URL . . . {requestURL}', 'info')
    
#     # Languange Pop up options will show
    wait = WebDriverWait(driver, 60)
    all_item_each_page = [driver.find_element(By.XPATH, '//div[@class="css-llwpbs"]')]
    total_item_per_page = len(all_item_each_page)
    print('each_item LENGTH', total_item_per_page )
    
    for item in all_item_each_page:
        print('item', item)
        anchor = item.find_element(By.TAG_NAME, 'a')
        current_url = driver.current_url
        print('current_url', current_url)
        print('acnhooorrr', anchor.get_attribute('href'))
        anchor.click()
#         driver.wait_for_page_load()
        wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="css-856ghu"]')))
        
        new_url = driver.current_url
        print('new_url', new_url)
        if current_url != new_url:
            [is_scrolled]=scrollFromToptoBottom(driver, 'pdp_comp-discussion_faq', True)
            getProductDetail(driver)
        
#         driver.back()
#         `Ctrl` + `Click` keys to open the link in a new tab.
#         anchor.send_keys(Keys.chord(Keys.CONTROL, Keys.RETURN))
#         driver.execute_script('window.open(arguments[0])', anchor.get_attribute('href'))
#         new_tab_window_handle = driver.window_handles[1]

        # Switch to the new tab.
#         current_window = driver.switch_to.window(new_tab_window_handle)
#         if current_window:
#             print('OPEN')


#     errorElement = driver.find_element(By.XPATH, '//div[@id="main-frame-error"]')
#     print('errorElement', errorElement)    
#     if errorElement:
#         raise NoSuchElementException
#         print_message('Oops, Something went wrong! reason: BLOCKED or TIMEOUT', 'danger', bold=True)
# #         driver.quit()
#     CHOOSE_LANGUAGE_BUTTON = driver.find_elements(By.XPATH, '//button[@class="shopee-button-outline shopee-button-outline--primary-reverse"]')
 
    


In [293]:
retry_count = 0
MAX_RETRY = 10
while retry_count < MAX_RETRY:
    try:
        scrapeTokopediaData()
        break
    except TimeoutException:
        print_message(f"TimeoutException occurred in main function (retry {retry_count + 1}/{MAX_RETRY}), retrying in 10 seconds...", 'warning')
        time.sleep(10)
        retry_count += 1
    except NoSuchElementException:
        print_message(f"NoSuchElementException occurred  in main function (retry {retry_count + 1}/{MAX_RETRY}), retrying in 10 seconds...", 'warning')
        time.sleep(10)
        retry_count += 1
else:
    print_message(f'Max retries reached {MAX_RETRY}. exiting...', 'danger', bold=True)

SCRAPE FUNCTION RUNNING ON PROXY 34.82.224.175:33333 . . .
ARGUMENT_ADDED
DRIVER_SETUP
REQUESTING_URL . . . None
each_item LENGTH 1
item <selenium.webdriver.remote.webelement.WebElement (session="0bf7525215f7d1e414ad7829805c3316", element="9A68880A0C92C03BEA5A5691484121B9_element_68")>
current_url https://www.tokopedia.com/search?st=&q=sony
acnhooorrr https://ta.tokopedia.com/promo/v1/clicks/8a-xgVY2gmUEH_rdopjhosyabm-xgVY789CBUsthbm-Orfua9fBjUstiHmUDUSz2Q31i6sJf6_KNoAJOHsJDUMoFyaUEH_UFHmFircYpq9z2Qfdi6sJDUMVDgaUEUSo7QuYDH3h5PaUDUMNOQ3-BrBY5gBYxgIHi6sUObm-XP3Oig9-wy3zp9R-BrZUEHs1DUMoxPVY2gIHi6BDOopKDHprpHmFpopjf9ZFiQBYsy3Njq3zxPcuwy3zpUstR6ZFiy3-wPcupPmUEUjdibm-FQRo79fVDgaUEUMzBgiUDUSgBrSo2Qfdi6i-fHiUDUMVj9RosQR-BUstO6_HXHAyOHAKf6AjFoAH7oZFirpowQcYSUstig9BGqMzUZMggQj2fgAo6QJBkQfBoe7BpZ3O6HcoD692qu7gN3_-Sq1Y2Z9P-q9P2y_-3o3ea69BqzsBE3_UN8u2_Z_g-qjV2_JoGP3Uao32q17jfZ3OqQJNk3_-oqjJ7_VPo8V2W39z6zchW_uHFoVBEZ92HuJ1a_7zSHjOJe9PquJ2W3uo-r7BW69BxufzFyMFNqO2yeMgxuOV2_fB-P7B2PfBiH72F3s-DPuKpeSBiH